<p>Similar to relational databases, the Spark DataFrame and Dataset APIs
and Spark SQL offer a series of join transformations: inner joins, outer joins, left
joins, right joins, etc. All of these operations trigger a large amount of data movement
across Spark executors.</p>

<h3>Five join strategiy</h3>
<ol>
    <li>The broadcast hash join (BHJ)</li>
    <li>Shuffle hahs join (SHJ)</li>
    <li>Shuffle sort merge join (SMJ)</li>
    <li>Broadcast nested loop join (BNLJ)</li>
    <li>Shuffle and replicated nested loop join (Castesian product)</li>
</ol>

# 2. Shuffle Hash Join (SHJ)

Shuffle Hash Join is divided into two steps:

1.      On the two tables were in accordance with the join keys re-zoning, that shuffle, the purpose is to have the same join keys value of the record assigned to the corresponding partition

2.       The corresponding partition in the data for the join, here first small table partition is constructed as a hash table, and then according to the large table recorded in the join keys value out to match.
3.       If you can't use broadcast join and your tables fairly big enough you use SHJ.
4.       One of the two tables are expected to be smaller than the other.
5.       Join keys might not be sortable.

# 3. Shuffle sort merge join (SMJ)

<p> By default Spark will use a <strong>broadcast join</strong> if the smaller data set is less than 10 MB. <br>
This configuration is set in spark.sql.autoBroadcastJoinThreshold; <br>
# By setting this value to -1 broadcasting can be disabled. <br>
    We use <strong>Sort Merge Joins if we are joining two big tables.</strong> This join scheme has two phases: a sort phase followed by a merge phase. If you use buckets on join columns performance will increase due to preordering the columns.</p>


In [1]:
! ls -l ~/datasets

total 139844
-rw-rw-r--. 1 train train     4556 Jul 21  2020 Advertising.csv
-rw-rw-r--. 1 train train   674856 Sep 24 19:59 Churn_Modelling.csv
drwxr-xr-x. 3 train train       96 Nov 19  2020 churn-telecom
-rw-rw-r--. 1 train train 41002480 Aug 27 19:45 Fire_Incidents.csv.gz
-rw-rw-r--. 1 train train 46401315 Aug 27 11:25 Hotel_Reviews.csv.gz
-rw-rw-r--. 1 train train     4611 Nov 20  2020 iris.csv
drwxr-xr-x. 2 train train      198 Jul 19 12:15 iris_parquet
-rw-rw-r--. 1 train train     4365 Aug 27 12:29 Mall_customers
-rw-rw-r--. 1 train train     4365 Aug 27 11:20 Mall_Customers.csv
drwxrwxr-x. 2 train train      133 Jul 23  2020 retail_db
drwxr-xr-x. 2 train train     4096 Jul 20 12:27 salary_avro
drwxr-xr-x. 2 train train     4096 Jul 20 12:29 salary_json
drwxr-xr-x. 2 train train     4096 Jul 20 12:28 salary_orc
-rw-rw-r--. 1 train train      592 Jul 22 10:04 simple_data.csv
-rw-rw-r--. 1 train train 40044293 Sep 19  2019 tmdb_5000_credits.csv
-rw-rw-r--. 1 train train  9317430 

In [1]:
import findspark
findspark.init("/opt/manual/spark")
from pyspark.sql import SparkSession, functions as F

In [2]:
spark = (
SparkSession.builder
    .appName("Joins")
    .master("local[2]")
    .config("spark.driver.memory","3g")
    .getOrCreate()
)

2022-09-24 21:55:19,251 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
# Disable broadcast hash join
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

In [4]:
spark.conf.get("spark.sql.autoBroadcastJoinThreshold")

'-1'

## Shuffle Sort Merge Join

In [5]:
order_items = spark.read.format("csv") \
.option("header", True) \
.option("inferSchema", True) \
.option("sep", ",") \
.load("file:///home/train/datasets/retail_db/order_items.csv")

In [6]:
order_items.limit(3).toPandas()

,orderItemName,orderItemOrderId,orderItemProductId,orderItemQuantity,orderItemSubTotal,orderItemProductPrice
0,1,1,957,1,299.98,299.98
1,2,2,1073,1,199.99,199.99
2,3,2,502,5,250.00,50.00


In [7]:
orders = spark.read.format("csv") \
.option("header", True) \
.option("inferSchema", True) \
.option("sep", ",") \
.load("file:///home/train/datasets/retail_db/orders.csv")

In [8]:
orders.limit(5).toPandas()

,orderId,orderDate,orderCustomerId,orderStatus
0,1,2013-07-25 00:00:00.0,11599,CLOSED
1,2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT
2,3,2013-07-25 00:00:00.0,12111,COMPLETE
3,4,2013-07-25 00:00:00.0,8827,CLOSED
4,5,2013-07-25 00:00:00.0,11318,COMPLETE


In [9]:
ssmerge_join_df = orders.join(order_items, 
                                orders.orderId == order_items.orderItemOrderId)

In [10]:
ssmerge_join_df.explain()

== Physical Plan ==
*(5) SortMergeJoin [orderId#50], [orderItemOrderId#17], Inner
:- *(2) Sort [orderId#50 ASC NULLS FIRST], false, 0
:  +- Exchange hashpartitioning(orderId#50, 200), ENSURE_REQUIREMENTS, [id=#79]
:     +- *(1) Filter isnotnull(orderId#50)
:        +- FileScan csv [orderId#50,orderDate#51,orderCustomerId#52,orderStatus#53] Batched: false, DataFilters: [isnotnull(orderId#50)], Format: CSV, Location: InMemoryFileIndex[file:/home/train/datasets/retail_db/orders.csv], PartitionFilters: [], PushedFilters: [IsNotNull(orderId)], ReadSchema: struct<orderId:int,orderDate:string,orderCustomerId:int,orderStatus:string>
+- *(4) Sort [orderItemOrderId#17 ASC NULLS FIRST], false, 0
   +- Exchange hashpartitioning(orderItemOrderId#17, 200), ENSURE_REQUIREMENTS, [id=#87]
      +- *(3) Filter isnotnull(orderItemOrderId#17)
         +- FileScan csv [orderItemName#16,orderItemOrderId#17,orderItemProductId#18,orderItemQuantity#19,orderItemSubTotal#20,orderItemProductPrice#21] Batched: fal

In [12]:
# As you from the plan spark sorts the join keys then executes join operations.

In [13]:
spark.stop()

# 4. Broadcast nested loop join

    Broadcast nested loop join - In nested join for each row of first data set is iterate over every row of other dataset which may degrade performance in join operation.But in certain situation like join keys are not fixed as well as the query is qualified as broadcastable or not, according to the data statistics (size or broadcast hint). If neither of them is evaluated as true and the join type is inner, the query is executed with CartesianProductExec. In this cases BroadcastNestedLoopJoinExec is taken. One of the places where nested loop join is used independently on the dataset size is cross join resulting on cartesian product. In this situation each row from the left table is returned together with every row from the right table, if there is no predicate defined. Apache Spark provides a support for such type of queries with org.apache.spark.sql.execution.joins.BroadcastNestedLoopJoinExec physical operator. It's used when neither broadcast hash join nor shuffled hash join nor sort merge join can be used to execute the join statement.